In [1]:
%%HTML
<button onclick="$('.input, .prompt, .output_stderr, .output_error, .output_result').toggle();">隐藏代码</button>

## 全局变量

In [64]:
spark= None
data1 = None
data2 = None

## 创建spark环境

In [73]:
from ipywidgets import *
from ipywidgets import HBox, Label
import numpy as np
import pandas as pd

from pyspark.mllib.recommendation import Rating
from pyspark.sql import SparkSession

from pyspark.mllib.recommendation import ALS


spark_master = widgets.Dropdown(
    options=['local', 'yarn', 'local[*]'],
    value='local[*]',
    description='spark master:',
    layout={'width': '300px'}
)

display(spark_master)

bt_creat_sc = widgets.Button(
    description='创建环境',
    layout={'width': '300px'}
)

def creat_sparksc(bt, master):
    global spark
    spark = SparkSession.builder.master(master).getOrCreate()
    print("创建spark环境完成 spark运行在", spark.sparkContext.master)
    bt_creat_sc.disabled = True
    bt.value = spark

bt_creat_sc.on_click(lambda x: creat_sparksc(x,spark_master.value))
display(bt_creat_sc)

def load_data(handle,data1_path,data2_path):

    data1 = spark.read.csv(data1_path.value)
    data1 = data1.rdd.map(lambda x:x._c0.split("\t")[:3])
    data1.cache()
    
    p_items = spark.read.csv(data2_path.value,sep="|")
    products = dict(p_items.select("_c0","_c1").toDF("id","product").toLocalIterator())

    print("数据总量",data1.count())
    print("用户数",data1.map(lambda x: x[0]).distinct().count())
    print("电影数",data1.map(lambda x: x[1]).distinct().count())
    print("数据加载完成 😊\n")
    handle.disabled = True
    data1_path.data = data1
    data2_path.data = products
    

data1_path = widgets.Text(
    value='./data/ml-100k/u.data',
    description='数据一',
)
data2_path = widgets.Text(
    value='./data/ml-100k/u.item',
    description='数据二',
)
display(data1_path)
display(data2_path)

bt_load_data = widgets.Button(
    description='加载数据',
    layout={'width': '300px'}
)
bt_load_data.on_click(lambda x:load_data(x,data1_path,data2_path))
display(bt_load_data)

rank_sel = widgets.IntSlider(
    description='rank',
    value=10,
)
iter_sel = widgets.IntSlider(
    value=10,
    description='迭代次数',
)
lambda_sel = widgets.FloatSlider(
    value=0.01,
    description='lambda',
    max=1,
    step=0.01,
    
)

def train_model(bt,ratings, rank, iterations, lambda_):
    print("开始训练")
    model = ALS.train(
        ratings=ratings, 
        rank=rank,
        iterations=iterations,
        lambda_=lambda_
    )
    print("训练完成")
    bt.value = model
    
bt_train_model = widgets.Button(
    description='训练模型',
    layout={'width': '300px'}
)
bt_train_model.on_click(lambda x:train_model(x, data1_path.data, rank_sel.value, iter_sel.value, lambda_sel.value))



display(rank_sel)
display(iter_sel)
display(lambda_sel)
display(bt_train_model)


display(HTML("<hr>"))

Dropdown(description='spark master:', index=2, layout=Layout(width='300px'), options=('local', 'yarn', 'local[…

Button(description='创建环境', layout=Layout(width='300px'), style=ButtonStyle())

Text(value='./ml-100k/u.data', description='数据一')

Text(value='./ml-100k/u.item', description='数据二')

Button(description='加载数据', layout=Layout(width='300px'), style=ButtonStyle())

...

IntSlider(value=10, description='迭代次数')

FloatSlider(value=0.01, description='lambda', max=1.0, step=0.01)

Button(description='训练模型', layout=Layout(width='300px'), style=ButtonStyle())

HTML(value='<hr>')

创建spark环境完成 spark运行在 local[*]
数据总量 100000
用户数 943
电影数 1682
数据加载完成 😊

开始训练
训练完成


In [76]:
import matplotlib.pyplot as plt
import numpy as np
user = widgets.Text(description='选择用户',value='1')
def f(user, model):
    def format_output(ratings):
        global data2_path
        for i in ratings:
            print("用户{0} 推荐电影{1}，推荐评分 {2:.2f}".format(i.user,data2_path.data[str(i.product)],i.rating))
    user 
    
    format_output(model.recommendProducts(user=int(user),num=5))
    

out = widgets.interactive_output(f, {'user': user, 'model': bt_train_model})

widgets.HBox([widgets.VBox([user]), out])
                                     
                                     
                                     